<a href="https://colab.research.google.com/github/jonhyuk0922/kaggle_study/blob/main/%5Bdacon%5D_%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사용 패키지

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import zipfile

In [116]:
#HYPER PARAMETERS

class CFG:
  emb_dim = 200

args = CFG

## 데이터 로드

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
local_zip = '/content/drive/MyDrive/HackerTon/데이콘_구내식당/235743_구내식당 식사 인원 예측 AI 경진대회_data.zip'

zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [117]:
train = pd.read_csv('/tmp/train.csv')
test = pd.read_csv('/tmp/test.csv')
submission = pd.read_csv('/tmp/sample_submission.csv')

## 데이터

In [118]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [119]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [120]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


## Word Embedding

In [121]:
df_all = pd.concat([train, test])

In [122]:
# Basic text preprocessing

def split_process(x, q):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_

In [123]:
# Get all combinations for training w2v (train + test)

food_combinations = []
for i in ['조식메뉴', '중식메뉴', '석식메뉴']:
    food_combinations += df_all[i].apply(lambda x: split_process(x, i)).to_list()

In [126]:
# Train or load w2v model

TRAIN_W2V = True
try:
    model = Word2Vec.load('food_embedding.model')
    print("Model loaded")
except:
    if TRAIN_W2V:
        print("Training w2v")
        model = Word2Vec(sentences=food_combinations, vector_size=args.emb_dim, window=7, min_count=0, workers=4, sg=0, epochs=5000)
        model.save('food_embedding.model')
    else:
        print("Model loading failed. Do not train.")

Training w2v


TypeError: ignored

In [125]:
# w2v demo
model.wv.most_similar('된장찌개')

NameError: ignored

## 요일을 숫자로 매핑

In [17]:
train['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

## 수치 데이터만 피쳐로 선택

In [18]:
x_train = train[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
y1_train = train['중식계']
y2_train = train['석식계']

x_test = test[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

In [93]:
model_dict = {
    'linear':LinearRegression(), 
    'rf':RandomForestRegressor(random_state=42,n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=42),
    'TF': tf.keras.models.Sequential([
            tf.keras.layers.Dense(units=64,activation='relu'),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.Dense(units=1)             
    ])
}

In [94]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm', 'TF'])

## 점심과 저녁을 식수 인원을 예측할 2개 모델 생성

In [95]:
model1 = model_dict['TF']
model2 = model_dict['TF']

In [96]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),loss=tf.losses.MeanSquaredError(), metrics = [tf.metrics.MeanAbsoluteError(), tf.metrics.RootMeanSquaredError()])
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),loss=tf.losses.MeanSquaredError(), metrics = [tf.metrics.MeanAbsoluteError(), tf.metrics.RootMeanSquaredError()])

## 학습

In [97]:
#tf 학습
model1.fit(x_train, y1_train,epochs=100,verbose=1)

Epoch 1/100
38/38 [==============================] - 1s 3ms/step - loss: 89873.0391 - mean_absolute_error: 232.9600 - root_mean_squared_error: 299.7883
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 40891.6562 - mean_absolute_error: 163.3940 - root_mean_squared_error: 202.2169
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 38085.1719 - mean_absolute_error: 158.3064 - root_mean_squared_error: 195.1542
Epoch 4/100
38/38 [==============================] - 0s 3ms/step - loss: 37727.4531 - mean_absolute_error: 157.4930 - root_mean_squared_error: 194.2356
Epoch 5/100
38/38 [==============================] - 0s 3ms/step - loss: 37406.5234 - mean_absolute_error: 156.9888 - root_mean_squared_error: 193.4077
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 37446.1562 - mean_absolute_error: 156.2335 - root_mean_squared_error: 193.5101
Epoch 7/100
38/38 [==============================] - 0s 3ms/step - loss: 38211.4023 - me

In [98]:
model2.fit(x_train, y2_train,epochs=100,verbose=1)

Epoch 1/100
38/38 [==============================] - 0s 3ms/step - loss: 33695.5898 - mean_absolute_error: 136.7238 - root_mean_squared_error: 183.5636
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 14093.3291 - mean_absolute_error: 85.8835 - root_mean_squared_error: 118.7153
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 13194.4385 - mean_absolute_error: 80.1982 - root_mean_squared_error: 114.8671
Epoch 4/100
38/38 [==============================] - 0s 3ms/step - loss: 13129.3564 - mean_absolute_error: 81.2347 - root_mean_squared_error: 114.5834
Epoch 5/100
38/38 [==============================] - 0s 3ms/step - loss: 12783.4561 - mean_absolute_error: 79.4771 - root_mean_squared_error: 113.0639
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 13033.3906 - mean_absolute_error: 80.4534 - root_mean_squared_error: 114.1639
Epoch 7/100
38/38 [==============================] - 0s 3ms/step - loss: 12892.1162 - mean_ab

In [92]:
model1.fit(x_train, y1_train)
model2.fit(x_train, y2_train)

38/38 [==============================] - 0s 3ms/step - loss: 29602.2598 - mean_absolute_error: 127.7489 - root_mean_squared_error: 172.0531


## 추론

In [99]:
pred1 = model1.predict(x_test)
pred2 = model2.predict(x_test)

## 결과 제출

In [100]:
submission['중식계'] = pred1
submission['석식계'] = pred2

In [101]:
submission.to_csv('baseline.csv', index=False)